In [1]:
import numpy as np
import pandas as pd

In [4]:
import streamlit as st
import pandas as pd
import plotly
import wordcloud
import matplotlib
import seaborn as sns
import numpy as np
import scipy
import sklearn

libraries = {
    'Streamlit': st.__version__,
    'Pandas': pd.__version__,
    'Plotly': plotly.__version__,
    'Wordcloud': wordcloud.__version__,
    'Matplotlib': matplotlib.__version__,
    'Seaborn': sns.__version__,
    'NumPy': np.__version__,
    'SciPy': scipy.__version__,
    'Scikit-learn': sklearn.__version__
}

libraries

{'Streamlit': '1.31.1',
 'Pandas': '2.2.1',
 'Plotly': '5.19.0',
 'Wordcloud': '1.9.3',
 'Matplotlib': '3.8.3',
 'Seaborn': '0.13.2',
 'NumPy': '1.26.4',
 'SciPy': '1.12.0',
 'Scikit-learn': '1.4.1.post1'}

In [2]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [3]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0,36,3.0,2.0,2,1,850.0,0.0,0.0,0.0,1,1,0.82
1,0,95,2.0,2.0,2,1,1226.0,1.0,0.0,0.0,1,2,0.95
2,0,103,2.0,2.0,1,1,1000.0,0.0,0.0,0.0,1,0,0.32
3,0,99,3.0,4.0,4,3,1615.0,1.0,0.0,1.0,0,2,1.60
4,0,5,2.0,2.0,1,3,582.0,0.0,1.0,0.0,0,2,0.48


In [ ]:
# one hot encode -> sector, balcony, agePossession, furnishing type, luxury category, floor category

In [4]:
X = df.drop(columns=['price'])
y = df['price']

In [7]:
!pip install scikit-learn

  Using cached scikit_learn-1.4.1.post1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.12.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.6 MB 1.7 MB/s eta 0:00:07
    --------------------------------------- 0.2/10.6 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.3/10.6 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.3/10.6 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.3/10.6 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.4/10.6 MB 1.4 MB/s eta 0:00:08
   - -------------------------------------- 0.5/10.6 MB 1.4 MB/s eta 0:00:08
   -- ------------------------------------- 0.6/10.6 MB 1.5 MB/s eta 0:00:07
   -- ------------------------------------- 0.7/10.6 MB 1.6 MB/s eta 0:00:07
   -- ------------------------------------- 0.7/10.6 MB 1.4 MB/s eta 0:00:08
   -- 

In [ ]:
import scikit-learn as sklearn

In [8]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [9]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [10]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [11]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [12]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel='rbf'))
])

In [13]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [14]:
scores.mean()

0.8845360715052786

In [15]:
scores.std()

0.014784881452420046

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [17]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [156]:
y_pred = pipeline.predict(X_test)

In [157]:
y_pred = np.expm1(y_pred)

In [158]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5324591082613233